# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import pandas as pd

In [2]:
df_images = pd.read_csv('Project_data/train.csv',header=None)

In [3]:
df_images.columns = ['filename']
df_images.columns

Index(['filename'], dtype='object')

In [4]:
df_images[['filename','action','label']] = df_images['filename'].str.split(';',expand=True)

In [5]:
df_images.label.value_counts()

3    137
1    137
0    136
2    130
4    123
Name: label, dtype: int64

In [35]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import keras
import warnings
warnings.filterwarnings("ignore")

We set the random seed so that the results don't vary drastically.

In [36]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [10]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 16#experiment with the batch size

In [15]:
len(train_doc)//batch_size

41

In [16]:
len(train_doc)//batch_size * batch_size

656

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [23]:
img_idx = list(range(0, 30, 2))
len(img_idx)

15

In [18]:
batch_size = 16
source_path = 'Project_data/train'
folder_list = train_doc
t = np.random.permutation(folder_list)
imgs = os.listdir(source_path+'/'+ t[0].split(';')[0])
imgs

['WIN_20180925_17_55_45_Pro_00006.png',
 'WIN_20180925_17_55_45_Pro_00008.png',
 'WIN_20180925_17_55_45_Pro_00010.png',
 'WIN_20180925_17_55_45_Pro_00012.png',
 'WIN_20180925_17_55_45_Pro_00014.png',
 'WIN_20180925_17_55_45_Pro_00016.png',
 'WIN_20180925_17_55_45_Pro_00018.png',
 'WIN_20180925_17_55_45_Pro_00020.png',
 'WIN_20180925_17_55_45_Pro_00022.png',
 'WIN_20180925_17_55_45_Pro_00024.png',
 'WIN_20180925_17_55_45_Pro_00026.png',
 'WIN_20180925_17_55_45_Pro_00028.png',
 'WIN_20180925_17_55_45_Pro_00030.png',
 'WIN_20180925_17_55_45_Pro_00032.png',
 'WIN_20180925_17_55_45_Pro_00034.png',
 'WIN_20180925_17_55_45_Pro_00036.png',
 'WIN_20180925_17_55_45_Pro_00038.png',
 'WIN_20180925_17_55_45_Pro_00040.png',
 'WIN_20180925_17_55_45_Pro_00042.png',
 'WIN_20180925_17_55_45_Pro_00044.png',
 'WIN_20180925_17_55_45_Pro_00046.png',
 'WIN_20180925_17_55_45_Pro_00048.png',
 'WIN_20180925_17_55_45_Pro_00050.png',
 'WIN_20180925_17_55_45_Pro_00052.png',
 'WIN_20180925_17_55_45_Pro_00054.png',


In [33]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = list(range(0, 30, 2)) #create a list of image numbers you want to use for a particular video
    x = len(img_idx)
    y = 60
    z = 60
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image_resize = imresize(image, (y, z))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = image_resize[:,:,0]/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image_resize[:,:,1]/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image_resize[:,:,2]/255#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        for folder_extra in range(batch_size):
            imgs = os.listdir(source_path+'/'+ t[(folder_extra + (num_batches*batch_size))%(len(folder_list))].split(';')[0])
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[(folder_extra + (num_batches*batch_size))%(len(folder_list))].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image_resize = imresize(image, (y, z))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = image_resize[:,:,0]/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image_resize[:,:,1]/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image_resize[:,:,2]/255#normalise and feed in the image
                    
            batch_labels[folder_extra, int( t[(folder_extra + (num_batches*batch_size))%(len(folder_list))].strip().split(';')[2])] = 1
        yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
            


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [22]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 5# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 5


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [25]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
batch_size = 16
x = 15
y = 60
z = 60
no_classes = 5
#write your model here
model = Sequential()
model.add(Conv3D(16, kernel_size = (3,3,3), activation='relu',kernel_initializer='normal', input_shape=(x,y,z,3)))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Conv3D(32, kernel_size = (3,3,3), activation='relu', kernel_initializer = 'normal'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Flatten())
model.add(Dense(128, activation = 'relu', kernel_initializer = 'normal'))
model.add(Dense(no_classes, activation = 'softmax'))

Instructions for updating:
Colocations handled automatically by placer.


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [28]:
learning_rate = 0.001
optimiser = keras.optimizers.Adam(lr=learning_rate)#write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_2 (Conv3D)            (None, 13, 58, 58, 16)    1312      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 6, 29, 29, 16)     0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 4, 27, 27, 32)     13856     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 2, 13, 13, 32)     0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 10816)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1384576   
_________________________________________________________________
dense_2 (Dense)              (None, 5)                

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [34]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [30]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=2, min_lr=0.000001, verbose=1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [31]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [37]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/5
Source path =  Project_data/val ; batch size = 16
Source path =  Project_data/train ; batch size = 16
42/42 [==============================] - 270s 6s/step - loss: 1.3155 - categorical_accuracy: 0.4598 - val_loss: 2.6921 - val_categorical_accuracy: 0.5625

Epoch 00001: val_loss improved from inf to 2.69208, saving model to model_init_2019-12-1816_32_05.068501/model-00001-1.31551-0.45982-2.69208-0.56250.h5
Epoch 2/5
42/42 [==============================] - 186s 4s/step - loss: 0.9247 - categorical_accuracy: 0.7068 - val_loss: 2.6194 - val_categorical_accuracy: 0.5536

Epoch 00002: val_loss improved from 2.69208 to 2.61939, saving model to model_init_2019-12-1816_32_05.068501/model-00002-0.92474-0.70685-2.61939-0.55357.h5
Epoch 3/5
42/42 [==============================] - 181s 4s/step - loss: 0.7676 - categorical_accuracy: 0.7619 - val_loss: 2.9164 - val_categorical_accuracy: 0.5893

Epoch 00003: val_loss did not improve from 2.61939
Epoch 4/5
42/42 [===========================